# Example analysis jupyter with running viewer with loupe
Here we show how to use feature of the data-spec-loupe alongside the numerical analysis to get easy insight into the data.

In [ ]:
import sys
import os
from pathlib import Path

%load_ext autoreload
%autoreload 2

# TMP find repo path to have access to repo test data and be able to import data spec
repo_path = Path(os.getcwd()).parent
sys.path.append(str(repo_path))

## Prepare data paths and load the numerical analysis data.

In [ ]:
import pandas as pd

tsv_path = repo_path / "tests" / "data" / "nano" / "data_nano.tsv"
config_path = repo_path / "tests" / "data" / "nano" / "data_config.yaml"
views_path = repo_path / "tests" / "data" / "nano" / "views_nano"

loaded_data = pd.read_csv(tsv_path)
loaded_data.head(3)

## Startup the data-spec-louse explorer

In [ ]:
import napari
from dataspecloupe.explorer import Explorer
explorer = Explorer(views_root=views_path, data_path=tsv_path)
explorer.set_config(config_path)
explorer.prepare_gui()

# here we can optionally pass only interesting subset of features
columns_to_explore = ['view', 'label', 'track_label', 'fov', 'timepoint', 'trench', 'trench_lineage', 'lineage_cellid', 'parent_lineage_cellid', 'cell_generation']
explorer.subset_data_columns(columns_to_explore)

napari_viewer = explorer.viewer
napari.run()

#### Move to the view from analysis

In [ ]:
explorer.set_view(0)

#### Mark objects from the selected data rows
Here we calculate the largest objects.

In [ ]:
# set selected column to the selected rows (assuming label column)
largest_ones = loaded_data.sort_values('geometry_Area', ascending=False).head(10)
largest_ones

In [ ]:
explorer.set_selected_objects(largest_ones)

Now the selection can be visualised by selecting SELECTED feature in feature visualisator.

There is option to provide `selected_value` to `set_selected_objects` and have SELECTED have multiple values representing various sets.
To clear up just provide empty list with `selected_value` used to fill it in.


#### New dataframe and be created and used in the explorer
Here we calculate for each track_label how many times it appeared.

In [ ]:
new_dataframe = loaded_data[["label", "track_label", "timepoint"]].copy()

new_dataframe["count_unique_timepoints"] = (
    new_dataframe.groupby("track_label")["timepoint"]
      .transform("nunique")
)
new_dataframe.head(3)

In [ ]:
explorer.set_features(new_dataframe)